<a href="https://colab.research.google.com/github/dionysakos/DCC_GARCH/blob/main/dcc_garch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance arch numpy pandas matplotlib scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 12.7 MB/s eta 0:00:00


In [47]:
import yfinance as yf
import numpy as np
import pandas as pd
from arch import arch_model
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots




So now we have to download  each asset's data from Yahoo Finance and calculate log returns.

In [50]:
def fetch_data(tickers, start_date, end_date):
 prices = yf.download(tickers,start=start_date,end=end_date)["Close"]
 returns = np.log(prices/prices.shift(1)).dropna()
 scaled_returns = returns*100
 return scaled_returns

scaled_returns = fetch_data(["AAPL","NVDA"],"2020-01-01","2024-06-01")
scaled_returns.head()

/tmp/ipython-input-3394240566.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers,start=start_date,end=end_date)["Close"]
[*********************100%***********************]  2 of 2 completed


Ticker,AAPL,NVDA
Date,,
2020-01-03,-0.976946,-1.613538
2020-01-06,0.793698,0.418485
2020-01-07,-0.471463,1.203414
2020-01-08,1.595833,0.187371
2020-01-09,2.101869,1.092270


Fit GARCH(1,1) for each ticker and get σ (conditional voltility at time t) and ε(shock/residual error of returns at time t)

In [51]:
from arch import arch_model
import pandas as pd

def fit_garch(scaled_returns):
    sigmas = pd.DataFrame(index=scaled_returns.index, columns=scaled_returns.columns, dtype=float)
    shocks = pd.DataFrame(index=scaled_returns.index, columns=scaled_returns.columns, dtype=float)

    for ticker in scaled_returns.columns:
        model = arch_model(scaled_returns[ticker],vol="Garch",p=1,q=1)
        res = model.fit(disp="off")
        sigmas[ticker] = res.conditional_volatility
        shocks[ticker] = res.resid

    return sigmas, shocks

sigmas, eps = fit_garch(scaled_returns)
sigmas.tail()


Ticker,AAPL,NVDA
Date,,
2024-05-24,1.562306,3.711619
2024-05-28,1.571469,3.574133
2024-05-29,1.509235,3.927741
2024-05-30,1.452090,3.702335
2024-05-31,1.405320,3.732414


In [52]:
print(sigmas.isna().sum())
print(eps.isna().sum())

Ticker
AAPL    0
NVDA    0
dtype: int64
Ticker
AAPL    0
NVDA    0
dtype: int64


Standardize residuals


In [53]:
z = eps/sigmas
z.dropna()
#Check the var for z, should be around 1 as it is a standardized r.v
print(z.var())

Ticker
AAPL    0.996819
NVDA    0.996749
dtype: float64


Calculate dynamic conditional correlations - DCC GARCH


In [54]:
def dcc_garch(z,a,b):
  if a<0 or b<0 or a+b>1:
    raise ValueError("Invalid values for a and b. No stability!")
  Z = z.values #convert into array
  T,N = Z.shape

  #T = number of days, N = number of assets, and Rt[t] is the NxN correlation matrix on day t.

  #Find unconditional/long term covariance of Z
  Qbar = np.cov(Z,rowvar = False) # Columns represent variables

  #Initialize
  Q = Qbar.copy()

  Rt = np.zeros((T,N,N))
  dates = z.index
  for t in range (T):
    if t>0:
     zprev = Z[t-1].reshape(N,1)
     Q = (1-a-b)*Qbar + a*np.dot(zprev,zprev.T) + b*Q

    d = np.sqrt(np.diag(Q))
    Dinv = np.diag(1.0/d)
    Rt[t]= np.dot(np.dot(Dinv,Q),Dinv)

  return Rt, dates

Rt, dates = dcc_garch(z,0.02,0.97)
Rt.shape




(1110, 2, 2)

In [46]:
Rt.min(),Rt.max()

(np.float64(0.22965127761450732), np.float64(1.0000000000000004))

Plot a pairwise dynamic correlation

In [55]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
import yfinance as yf

tickers = ["AAPL","NVDA"]

prices = yf.download(
    tickers,
    start="2020-01-01",
    end="2024-06-01",
    auto_adjust=True
)["Close"]

prices = prices.dropna()

returns_scaled = np.log(prices / prices.shift(1)).dropna() * 100


def plot_dcc_dashboard(returns_scaled, sigmas, dates, Rt, tickers, corr_pair=("AAPL","NVDA")):
    t1, t2 = corr_pair
    i = tickers.index(t1)
    j = tickers.index(t2)
    dcc_corr = Rt[:, i, j]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for tkr in tickers:
        fig.add_trace(
            go.Scatter(
                x=returns_scaled.index,
                y=returns_scaled[tkr],
                mode="lines",
                name=f"{tkr} Returns",
                opacity=0.7
            ),
            secondary_y=False
        )

    for tkr in tickers:
        fig.add_trace(
            go.Scatter(
                x=sigmas.index,
                y=sigmas[tkr],
                mode="lines",
                name=f"{tkr} GARCH Vol",
                line=dict(dash="dash"),
                opacity=0.9
            ),
            secondary_y=False
        )

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=dcc_corr,
            mode="lines",
            name=f"DCC Corr ({t1},{t2})",
            line=dict(width=3),
            opacity=0.6
        ),
        secondary_y=True
    )

    fig.update_layout(
        title="DCC-GARCH Analysis",
        xaxis_title="Date",
        height=650,
        width=1050,
        legend=dict(orientation="v"),
        hovermode="x unified"
    )

    fig.update_yaxes(title_text="Returns / Volatility", secondary_y=False)
    fig.update_yaxes(title_text="DCC Correlation", secondary_y=True, range=[-1, 1])

    fig.show()

tickers = list(returns_scaled.columns)
plot_dcc_dashboard(returns_scaled, sigmas, dates, Rt, tickers, corr_pair=("AAPL","NVDA"))


[*********************100%***********************]  2 of 2 completed
